# classify_dktc

predict and evaluation 해서 dktc 데이터를 dktc_exp, dktc_imp 로 결과까지 내어서 데이터 dktc_exp, dktc_imp를 csv 파일로 추출하는 것 까지 실행하기  
(threshold를 기준으로 일정 확률 사잇값을 ood: imp 레이블로 추출)

In [ ]:
# 라이브러리 import 및 주요 라이브러리 버전 확인 
import wandb
from wandb.keras import WandbCallback

import tensorflow as tf

import pandas as pd
import numpy as np  
import re
import pickle
from sklearn.model_selection import train_test_split

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

import json

import os
import warnings

warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# tf.logging.set_verbosity(tf.logging.ERROR)

from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam
from keras_radam.training import RAdamOptimizer
import encoding_korbert as enc

In [ ]:
model_extra = tf.keras.models.load_model('./saved/model_extra.h5')

In [ ]:
dktc_df = pd.read_csv("./data/merged_dktc_train.csv", usecols=['class','conversation']) #dktc_train (dktc는 train만 레이블이 달려 있어서 train만 써서 확인해봄)

In [ ]:
dktc_x, dktc_y = enc.load_data(dktc_df)

In [ ]:
tensor_dktc = tf.convert_to_tensor(np.array(dktc_x))

In [ ]:
probs = model_extra.predict(x=(tensor_dktc[0,:,:], tensor_dktc[1,:,:]), verbose=1)
#p는 확률값들의 배열 (0에 가까울수록 exp 1 에 가까울수록 not)

In [ ]:
np.set_printoptions(threshold=np.inf) #넘파이 출력 제어 , threshold=np.inf : 생략없이 출력

In [ ]:
probs #값을 확인하고 threshold_min, threshold_max 정하기

In [ ]:
probs.shape 

### dktc 데이터에서는 'not'이 없다.  
### 그래서 1에 가까울수록 not 이다.  
### 그러면 일정 값 이상의 확률을 가지는 데이터는 imp로 볼 수 있다.

In [ ]:
# threshold_min = 0.1
threshold_max = 0.85

indices = []
i=0
for p in probs:
    if threshold_max<p: 
        print(p)
        indices.append(i)
    i+=1
            
    #(0: exp, 1: not)

In [ ]:
indices

In [ ]:
np.set_printoptions(threshold=10)


In [ ]:
for i in indices:
    print(dktc_df['conversation'].iloc[i])
    print('====================================================')

In [ ]:
dktc_imp_df = dktc_df.loc[indices]
dktc_exp_df = dktc_df.loc[~dktc_df.index.isin(indices)]

In [ ]:
dktc_imp_df.to_csv('./data/dktc_imp.csv')
dktc_exp_df.to_csv('./data/dktc_exp.csv')